first we need to train a parametric-umap network for each dataset... (5 datasets x 2 dimensions)
For umap-learn, UMAP AE, Param. UMAP, PCA
- load dataset
- load network
- compute reconstruction MSE
- count time

In [1]:
# reload packages
%load_ext autoreload
%autoreload 2

### Choose GPU (this may not be needed on your computer)

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [3]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if len(gpu_devices)>0:
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

In [4]:
gpu_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import numpy as np
import pickle
import pandas as pd
import time
from umap import UMAP

In [6]:
from tfumap.umap import tfUMAP
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [7]:
from tqdm.autonotebook import tqdm

In [8]:
from tfumap.paths import ensure_dir, MODEL_DIR, DATA_DIR

In [9]:
output_dir = MODEL_DIR/'projections' 

In [10]:
reconstruction_acc_df = pd.DataFrame(columns = ['method_', 'dimensions', 'dataset', 'MSE', 'MAE', 'MedAE', 'R2'])

In [11]:
reconstruction_speed_df = pd.DataFrame(columns = ['method_', 'dimensions', 'dataset', 'embed_time', 'recon_time', 'speed', 'nex'])

### MNIST

In [12]:
dataset = 'fmnist'
dims = (28,28,1)

##### load dataset

In [13]:
from tensorflow.keras.datasets import fashion_mnist

# load dataset
(train_images, Y_train), (test_images, Y_test) = fashion_mnist.load_data()
X_train = (train_images/255.).astype('float32')
X_test = (test_images/255.).astype('float32')
X_train = X_train.reshape((len(X_train), np.product(np.shape(X_train)[1:])))
X_test = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

# subset a validation set
n_valid = 10000
X_valid = X_train[-n_valid:]
Y_valid = Y_train[-n_valid:]
X_train = X_train[:-n_valid]
Y_train = Y_train[:-n_valid]

# flatten X
X_train_flat = X_train.reshape((len(X_train), np.product(np.shape(X_train)[1:])))
X_test_flat = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
X_valid_flat= X_valid.reshape((len(X_valid), np.product(np.shape(X_valid)[1:])))
X_test = X_test.reshape((10000, 28,28,1))
print(len(X_train), len(X_valid), len(X_test))

50000 10000 10000


### AE 

##### 2 dims

In [14]:
load_loc = output_dir / dataset / 'autoencoder' 

In [15]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [16]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [17]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [18]:
X_recon = tf.nn.relu(decoder(encoder(X_test))).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [19]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [20]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['AE', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169


##### 64 dims

In [21]:
load_loc = output_dir / dataset / '64' / 'autoencoder' 

In [22]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [23]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [24]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [25]:
X_recon = tf.nn.relu(decoder(encoder(X_test))).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [26]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [27]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['AE', 64, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800


### Network

##### 2 dims

In [28]:
load_loc = output_dir / dataset / 'recon-network' 

In [29]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "network",
    batch_size = 100,
    dims = dims
)

In [30]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [31]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [32]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = encoder(X_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = decoder(z_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    recon_time = end_time - start_time
    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "network",
        2,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.13305453769862652
seconds:  0.12814724817872047
seconds:  0.20424984209239483
seconds:  0.010748082771897316
seconds:  0.2583285216242075
seconds:  0.005922432988882065
seconds:  0.27156236954033375
seconds:  0.007833173498511314
seconds:  0.26918722689151764
seconds:  0.005696220323443413
seconds:  0.27226319536566734
seconds:  0.010859478265047073
seconds:  0.26022934541106224
seconds:  0.010166091844439507
seconds:  0.2603007387369871
seconds:  0.00838213600218296
seconds:  0.27406904101371765
seconds:  0.01113332249224186
seconds:  0.2674020640552044
seconds:  0.005554484203457832



In [33]:
with tf.device('/CPU:0'):
    n_repeats = 10
    for i in tqdm(range(n_repeats)):
        start_time = time.monotonic()
        z_test = encoder(X_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        embed_time = end_time - start_time

        start_time = time.monotonic()
        x_test_recon = decoder(z_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        recon_time = end_time - start_time
        reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
            "network-cpu",
            2,
            dataset,
            embed_time,
            recon_time,
            embed_time + recon_time,
        len(X_test_flat)
        ]

seconds:  1.0309047307819128
seconds:  4.124898759648204
seconds:  0.5041633024811745
seconds:  4.098633393645287
seconds:  0.47247721068561077
seconds:  4.309320364147425
seconds:  0.514250073581934
seconds:  4.166348710656166
seconds:  0.4739694204181433
seconds:  3.9837655127048492
seconds:  0.469757542014122
seconds:  4.014683390036225
seconds:  0.469289168715477
seconds:  4.070288775488734
seconds:  0.46176254749298096
seconds:  3.954828090965748
seconds:  0.4588186349719763
seconds:  3.9887453634291887
seconds:  0.4632303472608328
seconds:  3.99966012313962



In [34]:
X_recon = tf.nn.relu(decoder(encoder(X_test))).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [35]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [36]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['network', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800
2,network,2,fmnist,0.026576,0.092112,0.046975,0.598707


##### 64 dims

In [37]:
load_loc = output_dir / dataset / '64' / 'recon-network' 

In [38]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [39]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [40]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [41]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = encoder(X_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = decoder(z_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    recon_time = end_time - start_time
    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "network",
        64,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.13145946897566319
seconds:  0.13911405578255653
seconds:  0.2000787202268839
seconds:  0.011582516133785248
seconds:  0.282875906676054
seconds:  0.009960962459445
seconds:  0.28806669637560844
seconds:  0.009043021127581596
seconds:  0.28226266987621784
seconds:  0.00632937066257
seconds:  0.28885403275489807
seconds:  0.008550066500902176
seconds:  0.2667786367237568
seconds:  0.007514305412769318
seconds:  0.28079548105597496
seconds:  0.011167613789439201
seconds:  0.2767299562692642
seconds:  0.0081565510481596
seconds:  0.2886369079351425
seconds:  0.006040764972567558



In [42]:
with tf.device("/CPU:0"):
    n_repeats = 10
    for i in tqdm(range(n_repeats)):
        start_time = time.monotonic()
        z_test = encoder(X_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        embed_time = end_time - start_time

        start_time = time.monotonic()
        x_test_recon = decoder(z_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        recon_time = end_time - start_time
        reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
            "network-cpu",
            64,
            dataset,
            embed_time,
            recon_time,
            embed_time + recon_time,
        len(X_test_flat)
        ]

seconds:  0.6627095099538565
seconds:  4.40268580429256
seconds:  0.4819801989942789
seconds:  4.323029559105635
seconds:  0.4746859222650528
seconds:  4.374355595558882
seconds:  0.47853116877377033
seconds:  4.303163517266512
seconds:  0.4763760343194008
seconds:  4.246158992871642
seconds:  0.47649998776614666
seconds:  4.249746851623058
seconds:  0.4941640719771385
seconds:  4.280312253162265
seconds:  0.47457018308341503
seconds:  4.244132958352566
seconds:  0.4700141344219446
seconds:  4.289180105552077
seconds:  0.46797144412994385
seconds:  4.328717058524489



In [43]:
reconstruction_speed_df

,method_,dimensions,dataset,embed_time,recon_time,speed,nex
0,network,2,fmnist,0.133055,0.128147,0.261202,10000
1,network,2,fmnist,0.204250,0.010748,0.214998,10000
2,network,2,fmnist,0.258329,0.005922,0.264251,10000
3,network,2,fmnist,0.271562,0.007833,0.279396,10000
4,network,2,fmnist,0.269187,0.005696,0.274883,10000
5,network,2,fmnist,0.272263,0.010859,0.283123,10000
6,network,2,fmnist,0.260229,0.010166,0.270395,10000
7,network,2,fmnist,0.260301,0.008382,0.268683,10000
8,network,2,fmnist,0.274069,0.011133,0.285202,10000
9,network,2,fmnist,0.267402,0.005554,0.272957,10000


In [44]:
X_recon = tf.nn.relu(decoder(encoder(X_test))).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [45]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [46]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['network', 64, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800
2,network,2,fmnist,0.026576,0.092112,0.046975,0.598707
3,network,64,fmnist,0.024062,0.087135,0.043900,0.624377


#### UMAP-learn

##### 2 dims

In [47]:
embedder = UMAP(n_components = 2, verbose=True)
z_umap = embedder.fit_transform(X_train_flat)

UMAP(dens_frac=0.0, dens_lambda=0.0, verbose=True)
Construct fuzzy simplicial set
Sat Jul 18 11:23:37 2020 Finding Nearest Neighbors
Sat Jul 18 11:23:38 2020 Building RP forest with 16 trees
Sat Jul 18 11:23:40 2020 parallel NN descent for 16 iterations
	 0  /  16
	 1  /  16
	 2  /  16
	 3  /  16
Sat Jul 18 11:23:58 2020 Finished Nearest Neighbor Search
Sat Jul 18 11:24:02 2020 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Sat Jul 18 11:24:29 2020 Finished embedding


In [48]:
x_test_samples= []
x_test_recon_samples= []
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    embed_time = end_time - start_time

    nex = 10 # it would take far too long to reconstruct the entire dataset
    samp_idx = np.random.randint(len(z_test),  size= nex)
    sample = np.array(z_test)[samp_idx]
    x_test_samples.append(samp_idx)
    start_time = time.monotonic()
    x_test_recon = embedder.inverse_transform(sample);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    recon_time = (end_time - start_time)*len(z_test)/nex

    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "umap-learn",
        2,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]
    x_test_recon_samples.append(x_test_recon)

	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  27.1037409119308
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  120.2995234746486
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  6.4580068327486515
	completed  0  /  1

In [49]:
x_recon = np.concatenate(x_test_recon_samples)

In [50]:
x_real = np.array(X_test_flat)[np.concatenate(x_test_samples)]

In [51]:

MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['umap-learn', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800
2,network,2,fmnist,0.026576,0.092112,0.046975,0.598707
3,network,64,fmnist,0.024062,0.087135,0.043900,0.624377
4,umap-learn,2,fmnist,0.034726,0.113416,0.068704,-9.198052


##### PCA

##### 2 dims


In [52]:
pca = PCA(n_components=2)
z = pca.fit_transform(X_train_flat)

In [53]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = pca.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = pca.inverse_transform(z_test);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    recon_time = (end_time - start_time)

    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "pca",
        2,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.02847796119749546
seconds:  0.02255089581012726
seconds:  0.02776527963578701
seconds:  0.024957740679383278
seconds:  0.027855820953845978
seconds:  0.02006395161151886
seconds:  0.03348085284233093
seconds:  0.025643808767199516
seconds:  0.027853911742568016
seconds:  0.020560503005981445
seconds:  0.03389427252113819
seconds:  0.02589048258960247
seconds:  0.027924375608563423
seconds:  0.019746070727705956
seconds:  0.03312550485134125
seconds:  0.024848608300089836
seconds:  0.027443762868642807
seconds:  0.020047606900334358
seconds:  0.03359130024909973
seconds:  0.024891868233680725



In [54]:
X_recon = pca.inverse_transform(pca.transform(X_test_flat))
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['pca', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800
2,network,2,fmnist,0.026576,0.092112,0.046975,0.598707
3,network,64,fmnist,0.024062,0.087135,0.043900,0.624377
4,umap-learn,2,fmnist,0.034726,0.113416,0.068704,-9.198052
5,pca,2,fmnist,0.046097,0.150541,0.117590,0.361385


##### 64 dims

In [55]:
pca = PCA(n_components=64)
z = pca.fit_transform(X_train_flat)

In [56]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = pca.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = pca.inverse_transform(z_test);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    recon_time = (end_time - start_time)

    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "pca",
        64,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.02187681384384632
seconds:  0.02169712632894516
seconds:  0.02140268310904503
seconds:  0.01710016466677189
seconds:  0.023456238210201263
seconds:  0.01743568480014801
seconds:  0.022197414189577103
seconds:  0.016955113038420677
seconds:  0.022531934082508087
seconds:  0.018048826605081558
seconds:  0.02657458186149597
seconds:  0.01763971522450447
seconds:  0.02170940861105919
seconds:  0.0181866604834795
seconds:  0.024162037298083305
seconds:  0.017588358372449875
seconds:  0.024210462346673012
seconds:  0.017385561019182205
seconds:  0.024035131558775902
seconds:  0.016995316371321678



In [57]:
X_recon = pca.inverse_transform(pca.transform(X_test_flat))
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['pca', 64, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800
2,network,2,fmnist,0.026576,0.092112,0.046975,0.598707
3,network,64,fmnist,0.024062,0.087135,0.043900,0.624377
4,umap-learn,2,fmnist,0.034726,0.113416,0.068704,-9.198052
5,pca,2,fmnist,0.046097,0.150541,0.117590,0.361385
6,pca,64,fmnist,0.010376,0.063705,0.038950,0.774832


### Save

In [58]:
#save_loc = DATA_DIR / 'reconstruction_speed' / (dataset + '.pickle')
#ensure_dir(save_loc)
#reconstruction_speed_df.to_pickle(save_loc)

In [59]:
save_loc = DATA_DIR / 'reconstruction_acc' / (dataset + '.pickle')
ensure_dir(save_loc)
reconstruction_acc_df.to_pickle(save_loc)